# VacationPy
---

## Starter Code to Import Libraries and Load the Weather and Coordinates Data

In [1]:
# Dependencies and Setup
import hvplot.pandas
import pandas as pd
import requests

# Import API key
from api_keys import geoapify_key

In [2]:
# Load the CSV file created in Part 1 into a Pandas DataFrame
city_data_df = pd.read_csv("output_data/cities.csv")

# Display sample data
city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,portland,45.5234,-122.6762,21.42,75,75,1.54,US,1723219118
1,1,shimoda,34.6667,138.9500,25.87,87,54,3.94,JP,1723219314
2,2,junin de los andes,-39.9504,-71.0694,10.25,62,86,2.63,AR,1723219129
3,3,hamilton,39.1834,-84.5333,28.45,58,0,3.09,US,1723219315
4,4,zyryanovsk,49.7386,84.2719,18.06,80,68,0.96,KZ,1723219315


---

### Step 1: Create a map that displays a point for every city in the `city_data_df` DataFrame. The size of the point should be the humidity in each city.

In [3]:
cities_map = city_data_df.hvplot.points(
    "Lng",
    "Lat",
    geo=True,
    tiles="OSM",
    frame_width=800,
    frame_height=600,
    size="Humidity", 
    scale=1,  
    color="Country",  
    hover_cols=["City", "Humidity", "Country"] 
)

# Display the map
cities_map

:Overlay
   .WMTS.I   :WMTS   [Longitude,Latitude]
   .Points.I :Points   [Lng,Lat]   (Country,Humidity,City)

### Step 2: Narrow down the `city_data_df` DataFrame to find your ideal weather condition

In [7]:
# Narrow down cities that fit criteria and drop any results with null values
ideal_cities_df = city_data_df[
    (city_data_df['Max Temp'] > 18) & 
    (city_data_df['Max Temp'] < 25) & 
    (city_data_df['Wind Speed'] < 8) & 
    (city_data_df['Cloudiness'] == 1)
]

# Drop any rows with null values
ideal_cities_df = ideal_cities_df.dropna()

# Display sample data
ideal_cities_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
64,64,atuona,-9.8000,-139.0333,24.69,69,1,7.05,PF,1666108268
276,276,tura,25.5198,90.2201,20.45,90,1,1.51,IN,1666108399
334,334,kailua,21.4022,-157.7394,22.47,89,1,1.18,US,1666108432
361,361,quelimane,-17.8786,36.8883,24.99,75,1,3.60,MZ,1666108448
450,450,samandag,36.0842,35.9771,21.85,80,1,2.31,TR,1666108498


### Step 3: Create a new DataFrame called `hotel_df`.

In [8]:
# Use the Pandas copy function to create DataFrame called hotel_df to store the city, country, coordinates, and humidity
hotel_df = ideal_cities_df[['City', 'Country', 'Lat', 'Lng', 'Humidity']].copy()

# Add an empty column, "Hotel Name," to the DataFrame so you can store the hotel found using the Geoapify API
hotel_df['Hotel Name'] = ""

# Display sample data
hotel_df.head()

,City,Country,Lat,Lng,Humidity,Hotel Name
64,atuona,PF,-9.8000,-139.0333,69,
276,tura,IN,25.5198,90.2201,90,
334,kailua,US,21.4022,-157.7394,89,
361,quelimane,MZ,-17.8786,36.8883,75,
450,samandag,TR,36.0842,35.9771,80,


### Step 4: For each city, use the Geoapify API to find the first hotel located within 10,000 metres of your coordinates.

In [15]:
# Set parameters to search for a hotel
radius = 10000
params = {
    "categories": "accommodation.hotel",
    "limit": 10,  
    "apiKey": geoapify_key,}  

# Print a message to follow up the hotel search
print("Starting hotel search")

# Iterate through the hotel_df DataFrame
for index, row in hotel_df.iterrows():
    # get latitude, longitude from the DataFrame
    latitude = row['Lat']
    longitude = row['Lng']
    
    # Add filter and bias parameters with the current city's latitude and longitude to the params dictionary
    params["filter"] = f"circle:{longitude},{latitude},{radius}"
    params["bias"] = f"proximity:{longitude},{latitude}"
    
    # Set base URL
    base_url = "https://api.geoapify.com/v2/places"


    # Make and API request using the params dictionaty
    name_address = requests.get(base_url, params=params)
    
    # Convert the API response to JSON format
    name_address = name_address.json()
    
    # Grab the first hotel from the results and store the name in the hotel_df DataFrame
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address["features"][0]["properties"]["name"]
    except (KeyError, IndexError):
        # If no hotel is found, set the hotel name as "No hotel found".
        hotel_df.loc[index, "Hotel Name"] = "No hotel found"
        
    # Log the search results
    print(f"{hotel_df.loc[index, 'City']} - nearest hotel: {hotel_df.loc[index, 'Hotel Name']}")

# Display sample data
hotel_df

Starting hotel search
atuona - nearest hotel: Pearl Resort
tura - nearest hotel: Hotel Polo Orchid
kailua - nearest hotel: No hotel found
quelimane - nearest hotel: Mabassa
samandag - nearest hotel: Almina Apart Hotel


,City,Country,Lat,Lng,Humidity,Hotel Name
64,atuona,PF,-9.8000,-139.0333,69,Pearl Resort
276,tura,IN,25.5198,90.2201,90,Hotel Polo Orchid
334,kailua,US,21.4022,-157.7394,89,No hotel found
361,quelimane,MZ,-17.8786,36.8883,75,Mabassa
450,samandag,TR,36.0842,35.9771,80,Almina Apart Hotel


### Step 5: Add the hotel name and the country as additional information in the hover message for each city in the map.

In [16]:
%%capture --no-display

# Configure the map plot
hotel_map = hotel_df.hvplot.points(
    'Lng', 'Lat', 
    geo=True, 
    size='Humidity', 
    color='Humidity', 
    cmap='viridis', 
    tiles='OSM', 
    frame_width=800, 
    frame_height=600, 
    hover_cols=['City', 'Country', 'Humidity', 'Hotel Name']
).opts(
    tools=['hover'],  # Enable hover tool
    width=800, 
    height=600
)

# Display the map
hotel_map

:Overlay
   .WMTS.I   :WMTS   [Longitude,Latitude]
   .Points.I :Points   [Lng,Lat]   (Humidity,City,Country,Hotel Name)